In [56]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [57]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [58]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [59]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [60]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [61]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-14 09:22:00,3211.21,3214.87,3211.01,3212.63,32.0,0
1,2024-11-14 09:23:00,3212.97,3213.43,3211.36,3212.10,27.0,1
2,2024-11-14 09:24:00,3212.01,3213.15,3209.48,3210.75,39.0,2
3,2024-11-14 09:25:00,3210.54,3210.90,3208.99,3210.28,20.0,3
4,2024-11-14 09:26:00,3210.22,3211.73,3210.13,3210.13,20.0,4


In [62]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

def check_crossing(df, col1, col2):
    # Calculate the difference between the two columns
    diff = df[col1] - df[col2]
    diff = diff/np.abs(diff)
    # Check if there is a sign change in the difference
    crossing = ((diff.shift(1) * diff) - 1)/-2
    
    return crossing

In [63]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
345,38.548154,16.550199,30.777743
346,37.412244,18.683735,29.622989
347,36.703004,17.718264,31.148179
348,36.295745,16.801577,31.899618


In [64]:
try: df = df.rename(columns={'EMA_60':'EMA_50'})
except: print("skip")

In [65]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [66]:
df.head()

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir
0,2024-11-14 09:22:00,3211.21,3214.87,3211.01,3212.63,32.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,NaN,NaN,NaN,0
1,2024-11-14 09:23:00,3212.97,3213.43,3211.36,3212.10,27.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3109.720000,NaN,0.02,0,NaN,NaN,NaN,0
2,2024-11-14 09:24:00,3212.01,3213.15,3209.48,3210.75,39.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3111.823000,NaN,0.02,0,NaN,NaN,NaN,0
3,2024-11-14 09:25:00,3210.54,3210.90,3208.99,3210.28,20.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.883940,NaN,0.02,0,NaN,NaN,NaN,0
4,2024-11-14 09:26:00,3210.22,3211.73,3210.13,3210.13,20.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3115.903661,NaN,0.02,0,NaN,NaN,NaN,0


In [67]:
df['r_ema_s_m'] = df['EMA_10'] / df['EMA_50']

In [68]:
len_data = 3

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_10', len_data)
    df.at[i + len_data-1, 'gradient_ema_10'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_14', len_data)
    df.at[i + len_data-1, 'gradient_rsi_14'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_50', len_data)
    df.at[i + len_data-1, 'gradient_ema_50'] = gradient  # Store the gradient in the row corresponding to n+4

In [69]:
# mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
# df['flag_ema_crossing'] = 0
# df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [70]:
df['flag_ema_crossing'] = check_crossing(df, 'EMA_10', 'EMA_50')

In [71]:
df.iloc[-20:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing
330,2024-11-14 14:52:00,3138.49,3138.49,3126.63,3130.24,297.0,330,3136.738922,3160.882796,37.715593,3103.614865,3148.3795,3193.144135,2.843662,0.297390,-11.645511,0.048298,-11.693809,3120.518687,NaN,0.06,0,42.592559,13.566024,31.831479,0,0.992362,-0.768005,-3.219124,-6.770,-1.122578,-0.0
331,2024-11-14 14:53:00,3130.26,3132.34,3126.66,3130.86,156.0,331,3135.670027,3159.705431,38.174702,3105.298904,3145.1740,3185.049096,2.535637,0.320515,-11.388988,0.243856,-11.632845,3122.156166,NaN,0.06,0,42.424126,13.054435,30.631078,0,0.992393,-1.256550,-1.494072,-3.455,-1.214045,-0.0
332,2024-11-14 14:54:00,3130.76,3137.15,3129.84,3129.84,118.0,332,3134.610022,3158.534238,37.682583,3108.259189,3142.0155,3175.771811,2.148704,0.319656,-11.139588,0.394605,-11.534193,3123.695396,NaN,0.06,0,41.536703,15.674339,29.109622,0,0.992426,-1.064450,-0.016505,-0.200,-1.174279,-0.0
333,2024-11-14 14:55:00,3130.07,3134.90,3125.31,3131.02,154.0,333,3133.957291,3157.455248,38.667616,3112.343347,3139.2000,3166.056653,1.711051,0.347710,-10.723111,0.648866,-11.371977,3125.142272,NaN,0.06,0,41.057254,14.646518,30.298278,0,0.992558,-0.856368,0.246457,0.080,-1.125092,-0.0
334,2024-11-14 14:56:00,3130.36,3132.59,3123.68,3123.92,109.0,334,3132.132329,3156.140141,35.075082,3114.229989,3136.7120,3159.194011,1.433476,0.215506,-10.840992,0.424788,-11.265780,NaN,3147.810000,0.02,1,40.733142,13.744722,29.559170,-1,0.992393,-1.238847,-1.303751,-2.960,-1.197049,-0.0
335,2024-11-14 14:57:00,3123.92,3129.40,3122.00,3122.58,174.0,335,3130.395542,3154.824057,34.425014,3115.660683,3134.5480,3153.435317,1.205106,0.183174,-10.916700,0.279264,-11.195964,NaN,3147.327400,0.04,0,40.559418,13.027313,29.201295,0,0.992257,-1.780875,-2.121301,-4.220,-1.315596,-0.0
336,2024-11-14 14:58:00,3122.54,3125.39,3120.36,3123.85,134.0,336,3129.205443,3153.609388,35.642442,3115.511504,3133.1405,3150.769496,1.125324,0.236499,-10.750297,0.356533,-11.106830,NaN,3146.314304,0.06,0,40.524386,12.547886,29.326534,0,0.992262,-1.463443,0.283680,-0.035,-1.265376,-0.0
337,2024-11-14 14:59:00,3123.86,3125.78,3108.37,3110.84,794.0,337,3125.866272,3151.932157,29.583314,3114.101114,3130.8605,3147.619886,1.070593,-0.097292,-11.535249,-0.342735,-11.192514,NaN,3144.757046,0.08,0,41.279858,11.034235,34.096474,0,0.991730,-2.264635,-2.420850,-5.870,-1.445950,-0.0
338,2024-11-14 15:00:00,3110.60,3111.69,3092.03,3093.33,927.0,338,3119.950586,3149.634033,23.735057,3106.928120,3127.9415,3148.954880,1.343592,-0.323559,-13.415594,-1.778464,-11.637130,NaN,3141.846082,0.10,0,42.724274,9.622616,40.367191,0,0.990576,-4.627429,-5.953693,-15.260,-1.987677,-0.0
339,2024-11-14 15:01:00,3093.32,3101.73,3089.78,3098.01,369.0,339,3115.961389,3147.609561,27.841014,3101.657884,3126.0890,3150.520116,1.563047,-0.074657,-14.362580,-2.180360,-12.182220,NaN,3136.864474,0.12,0,44.149601,8.879063,38.702864,0,0.989945,-4.952442,-0.871150,-6.415,-2.161298,-0.0


In [72]:
mask_ema_grad_pos = (df['gradient_ema_10']>0.05)
mask_ema_grad_neg = (df['gradient_ema_10']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_ema_grad_pos = (df['gradient_ema_50']>0.05)
mask_ema_grad_neg = (df['gradient_ema_50']<-0.05)
df['flag_grad_ema_50'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema_50'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema_50'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_14']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_14']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [73]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_10']>df['EMA_50']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_10']<df['EMA_50']), 'ema_short_above_or_below'] = -1

In [74]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_10'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_10'] / df['BBL_20_2.0']

# Strategy

To create a real-time trading strategy with specific technical indicator settings and strategies for entering and exiting trades, let's break down the components clearly:

### 1. **Technical Indicators**
Here's how we can set up the specific technical indicators:

#### Moving Averages
- **Short-Term Moving Average (MA)**: Use a 10-period MA. This is effective for spotting short-term trends and suitable for minute-by-minute trading data.
- **Long-Term Moving Average**: Use a 50-period MA to confirm the overall trend direction. This helps filter out the noise and provides a clearer direction.

#### Relative Strength Index (RSI)
- **Length**: A 14-period RSI is standard and works well in a variety of market conditions.
- **Overbought Threshold**: Set at 70 to indicate potential selling points.
- **Oversold Threshold**: Set at 30 to indicate potential buying points.

#### MACD (Moving Average Convergence Divergence)
- **Fast Line**: 12-period Exponential Moving Average (EMA)
- **Slow Line**: 26-period EMA
- **Signal Line**: 9-period EMA of the MACD line itself

### 2. **Entry and Exit Points**
- **Go Long (Buy)**: 
  - **Entry Point**: When the 10-period MA crosses above the 50-period MA and the RSI is below 70 but above 30, indicating neither overbought nor oversold conditions. Additionally, the MACD line should cross above the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back below the 50-period MA, or RSI approaches 70, indicating a potential overbought condition.
  
- **Go Short (Sell)**: 
  - **Entry Point**: When the 10-period MA crosses below the 50-period MA and the RSI is above 30 but below 70, indicating normal conditions. Also, the MACD line should cross below the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back above the 50-period MA, or RSI approaches 30, indicating a potential oversold condition.

### 3. **Stop Loss Criteria**
- **For Long Positions**: Place a stop loss below the most recent significant low or a fixed percentage, such as 2-3% below the entry price, depending on your risk tolerance.
- **For Short Positions**: Place a stop loss above the most recent significant high or a fixed percentage, such as 2-3% above the entry price.

### 4. **Risk Management**
- Consider the amount of capital at risk per trade, commonly 1-2% of your total trading capital to manage risk effectively.
- Use trailing stops to lock in profits as the market moves in your favor.

### 5. **Automation and Monitoring**
- Automate the strategy using a trading bot if possible, especially to handle the high-frequency nature of minute-by-minute trading.
- Regularly monitor performance and adjust parameters as necessary based on market conditions and strategy performance.

This setup gives you a robust framework for trading on a minute-by-minute basis, leveraging these technical indicators to make informed trading decisions. Adjust these settings based on backtesting results and your specific risk tolerance.

# Buy Signal

In [75]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)&(df['flag_grad_ema']>0)
mask_le2 = (df['MACDh_12_26_9']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_14']<70)&(df['RSI_14']>30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close']) & (df['psar_flip_dir']>0)
mask_le6 = (df['RSI_14']<40)
mask_le7 = (df['flag_grad_ema']>=0)

In [76]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_le4, 'rsi_30_to_70'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [77]:
df['buy_signal'] = np.nan
# df.loc[(mask_le1 & mask_le4) | (mask_le5 & mask_le4 & mask_le2) | (mask_le2 & mask_le6 & mask_le3), 'long_entry'] = 1
df.loc[(mask_le1 & mask_le4) | (mask_le6 & mask_le7), 'buy_signal'] = 1

In [78]:
# df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'rsi_30_to_70', 'PSAR_bellow_close']].sum(axis=1)

# Sell signal

In [79]:
mask_lex1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_lex2 = (df['RSI_14']>55)
mask_lex3 = (df['psar_flip_dir'] == -1)
mask_lex4 = (df['flag_grad_ema']<0)
mask_lex5 = (df['MACDh_12_26_9']<0)

In [80]:
df['ema_crossing_neg'] = 0
df.loc[mask_lex1, 'ema_crossing_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_lex5, 'macd_neg'] = 1

In [81]:
df['sell_signal'] = np.nan
df.loc[(mask_lex1) | (mask_lex2 & mask_lex4), 'sell_signal'] = 1

# Bullish / bearish confirmation

In [82]:
mask_bulber = (df['ADX_14']>=25)
mask_bul = (df['DMP_14']>=25)
mask_ber = (df['DMN_14']>=25)

df['trend_confirm'] = 0
df.loc[mask_bulber & mask_bul, 'trend_confirm'] = 1
df.loc[mask_bulber & mask_ber, 'trend_confirm'] = -1

# Over-sold/buy Marker

In [83]:
mask_os1 = (df['RSI_14']<=20)
mask_os2 = (df['r_close_bbl']<=1.000)
mask_ob1 = (df['RSI_14']>=80)
mask_ob2 = (df['r_close_bbu']>=1.000)

In [84]:
df['oversold_confirm'] = 0
df.loc[mask_os1|mask_os2, 'oversold_confirm'] = 1
df.loc[mask_ob1|mask_ob2, 'oversold_confirm'] = -1

# Check data

In [85]:
df.to_clipboard()

In [86]:
series_crypto = df.iloc[-1:][['oversold_confirm', 'trend_confirm', 'sell_signal', 'buy_signal']].sum()

In [88]:
string_f = f"Etherium: \n\n{series_crypto}"

print(string_f)

Etherium: 

oversold_confirm    0.0
trend_confirm      -1.0
sell_signal         0.0
buy_signal          0.0
dtype: float64
